Restarted aspire (Python 3.8.18)

In [4]:
import aspire
import numpy as np
import pandas as pd
from aspire.operators import RadialCTFFilter
from aspire.source.simulation import Simulation
from aspire.volume import LegacyVolume, Volume
from utils import volsCovarEigenvec
import time
from covar_estimation import im_stack_backward
import torch
# Specify parameters
img_size = 128  # image size in square
num_imgs = 2000  # number of images
dtype = np.float32

rank = 4
c = rank + 1
vols = LegacyVolume(
    L=img_size,
    C=c,
    dtype=dtype,
).generate()
vols -= np.mean(vols,axis=0)
class DummyFilter():
    def __init__(self,const):
        self.const = const
    def evaluate_grid(self,L):
        return np.ones((L,L)) * self.const
sim = Simulation(
    unique_filters=[RadialCTFFilter(defocus=d) for d in np.linspace(1.5e4, 2.5e4, 7)],
    #unique_filters = [DummyFilter(0.1)],
    n=num_imgs,
    vols=vols,
    dtype=dtype,
    amplitudes=1,
    offsets = 0
)

vectorsGD = torch.tensor(volsCovarEigenvec(vols),requires_grad = False)



2024-04-16 15:09:23,454 INFO [aspire.source.image] Creating Simulation with 2000 images.


In [2]:
%load_ext autoreload
%autoreload 2

from covar_sgd import CovarDataset,Covar,CovarTrainer
cds = CovarDataset(sim,vectorsGD = vectorsGD)
covar = Covar(resolution=img_size,rank=rank)#,vectors=vectorsGD.reshape((rank,img_size,img_size,img_size))/cds.im_norm_factor)
batch_size = 4
learning_rate = 1e-4
momentum = 0.9
reg = 1e-6 
gamma_lr = 0.8
gamma_reg = 0.8
kwargs_dict = {'max_epochs' : 10, 'lr' : learning_rate,'momentum' : momentum,'optim_type' : 'SGD','reg' : reg,'gamma_lr': gamma_lr,'gamma_reg' : gamma_reg}
#kwargs_dict = {'max_epochs' : 10, 'lr' : 1e-8,'momentum' : momentum,'optim_type' : 'Adam','reg' : reg,'gamma_lr': gamma_lr,'gamma_reg' : gamma_reg}
device = torch.device('cuda:0')
dataloader = torch.utils.data.DataLoader(cds,batch_size = batch_size,shuffle = False)#,collate_fn=dataset_collate)


In [ ]:
torch.manual_seed(0)
from covar_distributed import trainParallel

trainParallel(covar,cds,num_gpus = 8,batch_size = batch_size,**kwargs_dict)



In [3]:
torch.manual_seed(0)
trainer = CovarTrainer(covar,dataloader,device)
trainer.train(**kwargs_dict)

Epoch 0 , cost value : 2.00e+18,  cosine sim : 0.00, frobenium norm error : 7.16e+12:  15%|█▌        | 76/500 [00:08<00:42, 10.06it/s]

KeyboardInterrupt: 

In [ ]:
%load_ext autoreload
%autoreload 2
from covar_analyzer import CovarAnalyzer
import torch
c = CovarAnalyzer.load('data/tmp2/results.csv')

#c.plotCosineSim()
#c.plotWeightedCosineSim()
c.plotFroErr()




In [ ]:
import pickle
from projection_funcs import centered_fft2,vol_forward
x = pickle.load(open('data/pts.bin','rb'))
projs = x['projs'][0].reshape((rank,img_size,img_size))
fft_projs = aspire.image.Image(np.abs(centered_fft2(torch.tensor(projs)).numpy()))
projs = aspire.image.Image(projs)


projs.show()
fft_projs.show()

In [ ]:
from nufft_plan import NufftPlan
d = torch.device('cuda:1')
plan = NufftPlan((img_size,)*3,batch_size = rank,gpu_device_id = d.index,gpu_method=1,gpu_sort = 0)
plan.setpts(torch.tensor(x['pts']).to(d))
vols = torch.tensor(x['vols']).to(d)
proj_vols = vol_forward(vols,plan)
p = aspire.image.Image(proj_vols.cpu().numpy())
p.show()

In [ ]:

p = (torch.remainder(cds.pts_rot + torch.pi , 2 * torch.pi) - torch.pi) - torch.tensor(x['pts'])
#torch.tensor(x['pts']).shape
ind = torch.argmin(torch.norm(p.reshape((2000,-1)),dim=1))
aspire.utils.Rotation.from_matrix((sim.rotations[ind])).as_rotvec()



In [ ]:
b = fft_projs.asnumpy()[1].reshape(1,-1)
a = np.argmax(fft_projs.asnumpy()[1])
b[0,a]
x['pts'][:,a]